In [ ]:
# hide

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp clients

<IPython.core.display.Javascript object>

In [ ]:
from will_it_saturate.clients import BaseClient
from will_it_saturate.registry import register_model

<IPython.core.display.Javascript object>

In [ ]:
# export

import os
import math
import time
import httpx
import asyncio
import aiohttp
import subprocess

from pathlib import Path
from multiprocessing import Pool
from multiprocessing import set_start_method

# from will_it_saturate.old_core import Benchmark
from will_it_saturate.servers import BaseServer

<IPython.core.display.Javascript object>

In [ ]:
# os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"] = "YES"
# set_start_method("fork")
# print(os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"])

<IPython.core.display.Javascript object>

## Caveats

On macOS increase open file limit with:

```
ulimit -n 2048
```

Before starting the fastAPI Server with:

```
uvicorn will_it_saturate.fastapi.main:app --reload
```

It's not really possible to test forked client from this notebook. I don't know why. It works in the 03_run_benchmark script. Here I have to set_start_method("fork") and other ugly stuff.

In [ ]:
# dont_test

byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte

# file_sizes = [10 ** 7, 10 ** 6]
file_sizes = [10 ** 7, 10 ** 6, 10 ** 5]
# file_sizes = [10 ** 7]

# benchmark = Benchmark(
#     bandwidth=bandwidth,
#     duration=3,
#     file_sizes=file_sizes,
# )
# benchmark.create_epochs()

<IPython.core.display.Javascript object>

In [ ]:
# export


# just here because of broken nbdev confusing lua with python
counter = 0
request = None


@register_model
class HttpxClient(BaseClient):
    async def measure_server(self, epoch):
        print("measure server")
        print(epoch.urls[0])
        max_connections = min(epoch.number_of_connections, 50)
        limits = httpx.Limits(
            max_keepalive_connections=10, max_connections=max_connections
        )
        timeout = httpx.Timeout(30.0, connect=60.0)
        start = time.perf_counter()
        async with httpx.AsyncClient(limits=limits, timeout=timeout) as client:
            responses = await asyncio.gather(*[client.get(url) for url in epoch.urls])
        elapsed = time.perf_counter() - start
        print("done: ", elapsed)
        print("responses status: ", responses[0].status_code)
        return elapsed, responses

    def measure_in_new_process(self, epoch):
        print("new process")
        elapsed, responses = asyncio.run(self.measure_server(epoch))
        self.verify_checksums(epoch, responses)
        return elapsed

    def measure(self, epoch):
        print("measure")
        with Pool(1) as p:
            [result] = p.map(self.measure_in_new_process, [epoch])
        return result




# def run_httpx():
#     byte = 8
#     gigabit = 10 ** 9
#     bandwidth = gigabit / byte
# 
#     # file_sizes = [10 ** 7, 10 ** 6]
#     # file_sizes = [10 ** 7, 10 ** 6, 10 ** 5]
#     file_sizes = [10 ** 7]
# 
#     benchmark = Benchmark(
#         bandwidth=bandwidth,
#         duration=3,
#         file_sizes=file_sizes,
#         servers=[BenchmarkServer(name="uvicorn")],
#         clients=[HttpxClient(name="httpx")],
#     )
#     benchmark.create_rows()
#     benchmark.run()
#     print(benchmark.results_frame)
    



<IPython.core.display.Javascript object>

In [ ]:
# dont_test

# client = HttpxClient()
# elapsed, responses = await client.measure_server(benchmark.epochs[0])
# print(elapsed)

<IPython.core.display.Javascript object>

## aiohttp

In [ ]:
# export


class AioHttpResponse:
    def __init__(self, url, content):
        self.url = url
        self.content = content


@register_model
class AioHttpClient(BaseClient):
    async def fetch_page(self, session, url):
        async with session.get(url) as response:
            content = await response.read()
            return AioHttpResponse(url, content)

    async def measure_server(self, epoch):
        print("measure server")
        print(epoch.urls[0])
        urls = epoch.urls
        max_connections = min(epoch.number_of_connections, 200)
        conn = aiohttp.TCPConnector(limit=max_connections)
        responses = []
        start = time.perf_counter()
        async with aiohttp.ClientSession(connector=conn) as session:
            tasks = [asyncio.create_task(self.fetch_page(session, url)) for url in urls]
            responses = await asyncio.gather(*tasks)
        elapsed = time.perf_counter() - start
        return elapsed, responses

    def measure_in_new_process(self, epoch):
        elapsed, responses = asyncio.run(self.measure_server(epoch))
        self.verify_checksums(epoch, responses)
        return elapsed

    def measure(self, epoch):
        with Pool(1) as p:
            [result] = p.map(self.measure_in_new_process, [epoch])
        return result

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

client = AioHttpClient()
elapsed, responses = await client.measure_server(benchmark.epochs[0])
print(elapsed)

## wrk

In [ ]:
# export


@register_model
class WrkClient(BaseClient):
    connections: int = 20
    # set duration to two minutes since it is 10 seconds by default and kills the benchmark
    duration: int = 120
    threads: int = 1
    host: str = "localhost"
    port: str = "8000"

    def create_urls_string(self, epoch):
        urls = []
        for bf in epoch.files:
            urls.append(f'    {{path = "/{bf.path}"}},')
        return "\n".join(urls)

    def create_lua_script(self, epoch):
        requests_head = "requests = {"
        requests_tail = "}"
        lua_body = """
print(requests[1])

if #requests <= 0 then
  print("multiplerequests: No requests found.")
  os.exit()
end

print("multiplerequests: Found " .. #requests .. " requests")

counter = 1
request = function()
  -- Get the next requests array element
  local request_object = requests[counter]

  -- Increment the counter
  counter = counter + 1

  -- If the counter is longer than the requests array length -> stop and exit
  if counter > #requests then
    wrk.thread:stop()
    os.exit()
  end

  -- Return the request object with the current URL path
  return wrk.format(request_object.method, request_object.path, request_object.headers, request_object.body)
end
        """
        urls = self.create_urls_string(epoch)
        lua = "\n".join([requests_head, urls, requests_tail, lua_body])
        with Path(f"wrk.lua").open("w") as f:
            f.write(lua)

    def run_wrk(self):
        kwargs = {"capture_output": True, "text": True}
        start = time.perf_counter()
        command = [
            "wrk",
            "-d",
            str(self.duration),
            "-c",
            str(self.connections),
            "-t",
            str(self.threads),
            "-s",
            "wrk.lua",
            f"http://{self.host}:{self.port}",
        ]
        print("command: ", " ".join(command))
        output = subprocess.run(
            command,
            **kwargs,
        )
        elapsed = time.perf_counter() - start
        return elapsed

    def measure(self, epoch):
        print("measure? wtf?")
        self.create_lua_script(epoch)
        elapsed = self.run_wrk()
        return elapsed

<IPython.core.display.Javascript object>

## Wrk CLI command

```shell
time wrk -d 30 -c 20 -t 1 -s wrk.lua http://staging.wersdoerfer.de:5001
```

In [ ]:
%%time
# dont_test
kwargs = {"capture_output": True, "text": True}
output = subprocess.run(["wrk", "-c20", "-t1", "-d2", "-s", "wrk.lua", "http://localhost:8000"], **kwargs)
# output = subprocess.run(["wrk", "-d2", "http://localhost:8000"], **kwargs)

In [ ]:
# dont_test
print(output.stdout)

In [ ]:
# dont_test

client = WrkClient()
elapsed = client.measure(benchmark.epochs[0])
print(elapsed)

In [ ]:
# hide
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_index.ipynb.
Converted 01_host.ipynb.
Converted 02_file.ipynb.
Converted 03_registry.ipynb.
Converted 04_epochs.ipynb.
Converted 10_servers.ipynb.
Converted 11_views_for_fastapi_server.ipynb.
Converted 12_views_for_django_server.ipynb.
Converted 15_servers_started_locally.ipynb.
Converted 16_servers_started_via_docker.ipynb.
Converted 20_clients.ipynb.
Converted 21_benchmark_client_implementations.ipynb.
Converted 30_control_server.ipynb.
Converted 31_control_client.ipynb.
Converted 32_control_cli.ipynb.
Converted 40_results.ipynb.
Converted 41_repositories.ipynb.
Converted 42_sqlite_repository.ipynb.
Converted 50_benchmark_without_benchmark.ipynb.
Converted 51_benchmark_remote_control_server.ipynb.
Converted 52_benchmark_from_config.ipynb.
Converted 60_legacy.ipynb.
Converted 61_run_benchmark.ipynb.


<IPython.core.display.Javascript object>